# Week 4: Milestone 1 – Smart Tracking System Blockchain Ledger (Draft)

Follow these steps to verify Python↔Ganache↔Smart Contract connection.

1. Setup and Connection

1. **Open Jupyter Notebook**  
2. **Verify Ganache** is running (Desktop App “Quickstart Ethereum” or `ganache-cli`).  
3. **Check RPC port** (default is `7545` in Ganache Desktop).

Below we’ll connect Python → Ganache → our IoTDataStorage contract.

In [1]:
# Cell 1: imports & connection
from web3 import Web3
import json, os

# 1️⃣ Point to your Ganache RPC (update port if needed)
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# 2️⃣ Test connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


2. Load Contract ABI and Address

🔗 Now we load the **ABI** exported (in `/artifacts/IoTDataStorageABI.json`) and the **deployed address** from Remix.

In [2]:
# Cell 2: load ABI + contract
abi_path = os.path.join("..", "artifacts", "IoTDataStorageABI.json")
with open(abi_path) as f:
    abi = json.load(f)

# Replace with actual deployed address - convert to checksum format
contract_address_raw = "0x35a63ff4afbc8a5c3dc63fc893fbe5f2f0999f6e"
contract_address = Web3.to_checksum_address(contract_address_raw)

contract = web3.eth.contract(address=contract_address, abi=abi)

# Use the Ganache account that has a balance
web3.eth.default_account = web3.eth.accounts[1]

print(f"▶️ Using account {web3.eth.default_account}")
print(f"✅ Loaded contract at {contract_address}")

# Optional: Test the connection
try:
    total_package_records = contract.functions.getTotalPackages().call()
    print(f"📊 Current total records: {total_package_records}")
    print("🎉 Contract connection successful!")
except Exception as e:
    print(f"❌ Error testing contract connection: {e}")

▶️ Using account 0xE8128dc4B3Ac6bd4Bd04A269fd9581A0214ADD17
✅ Loaded contract at 0x35A63Ff4aFbC8A5c3dC63fc893FbE5F2f0999F6E
📊 Current total records: 0
🎉 Contract connection successful!


3. Read-Only Call: getTotalRecords()

Let’s check that our contract is responding:

In [3]:
# Cell 3: read-only call
total_events = contract.functions.getTotalEvents().call()
print(f"🔢 Total stored entries on‐chain: {total_events}")

🔢 Total stored entries on‐chain: 0


4. Write a dummy entry

Store one dummy IoT record (TEST001) to prove transactions work.

In [4]:
# Cell 4: Store three package entries with realistic city names
import datetime
from web3 import Web3
import random

# First, make sure we have a valid account
print("Available accounts:")
for i, account in enumerate(web3.eth.accounts):
    balance = web3.eth.get_balance(account)
    print(f"  [{i}] {account} - Balance: {web3.from_wei(balance, 'ether')} ETH")

# Set the account explicitly (use account[0] which is usually the deployer)
sender_account = web3.eth.accounts[0]  # or web3.eth.accounts[1] if you prefer
print(f"\n▶️ Using account: {sender_account}")

# Check account balance
balance = web3.eth.get_balance(sender_account)
balance_eth = web3.from_wei(balance, 'ether')
print(f"💰 Account balance: {balance_eth} ETH")

if balance_eth < 0.1:
    print("⚠️ Warning: Account balance is low. You may need more ETH for gas fees.")
    print("💡 In Ganache, you can restart to reset balances or use a different account.")

# Helper function to convert datetime string to timestamp
def datetime_to_timestamp(datetime_str):
    if datetime_str and datetime_str.strip():
        dt = datetime.datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
        return int(dt.timestamp())
    return 0

# Generate realistic package data (aligned with the fixed data generator)
def generate_realistic_package_data():
    """Generate realistic package data with proper city names"""

    # Realistic US cities
    us_cities = [
        "New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia",
        "San Antonio", "San Diego", "Dallas", "San Jose", "Austin", "Jacksonville",
        "Fort Worth", "Columbus", "Charlotte", "San Francisco", "Indianapolis",
        "Seattle", "Denver", "Washington", "Boston", "El Paso", "Nashville",
        "Detroit", "Oklahoma City", "Portland", "Las Vegas", "Memphis", "Louisville",
        "Baltimore", "Milwaukee", "Albuquerque", "Tucson", "Fresno", "Sacramento",
        "Kansas City", "Mesa", "Virginia Beach", "Atlanta", "Colorado Springs",
        "Omaha", "Raleigh", "Miami", "Oakland", "Minneapolis", "Tulsa", "Cleveland"
    ]

    # Realistic US states
    us_states = [
        "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
        "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
        "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
        "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
        "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
        "New Hampshire", "New Jersey", "New Mexico", "New York",
        "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
        "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
        "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
        "West Virginia", "Wisconsin", "Wyoming"
    ]

    package_types = ["Express", "Standard", "Priority", "Overnight", "Refrigerated", "Fragile"]
    carriers = ["FedEx", "UPS", "DHL", "USPS", "Amazon Logistics"]
    statuses = ["In Transit", "Processing", "Out for Delivery", "Delivered", "Exception", "Pending"]

    packages = []

    for i in range(3):
        carrier = random.choice(carriers)
        tracking_prefix = carrier[:3].upper()
        tracking_number = f"{tracking_prefix}{random.randint(100000000, 999999999)}"

        origin_city = random.choice(us_cities)
        origin_state = random.choice(us_states)
        dest_city = random.choice(us_cities)
        dest_state = random.choice(us_states)

        # Ensure origin and destination are different
        while dest_city == origin_city and dest_state == origin_state:
            dest_city = random.choice(us_cities)
            dest_state = random.choice(us_states)

        package = {
            "tracking_number": tracking_number,
            "package_type": random.choice(package_types),
            "carrier": carrier,
            "weight_kg": round(random.uniform(0.5, 50.0), 2),
            "length_cm": round(random.uniform(10, 100), 2),
            "width_cm": round(random.uniform(10, 100), 2),
            "height_cm": round(random.uniform(5, 100), 2),
            "origin_city": origin_city,
            "origin_state": origin_state,
            "origin_zip": f"{random.randint(10000, 99999)}",
            "destination_city": dest_city,
            "destination_state": dest_state,
            "destination_zip": f"{random.randint(10000, 99999)}",
            "current_status": random.choice(statuses),
            "estimated_delivery": "2025-06-15 14:30:00",  # Future date
            "signature_required": random.choice([True, False])
        }

        # Calculate volume
        package["volume_cm3"] = round(
            package["length_cm"] * package["width_cm"] * package["height_cm"], 2
        )

        # Generate description
        package["description"] = (
            f"{package['package_type']} package weighing {package['weight_kg']}kg, "
            f"dimensions: {package['length_cm']}x{package['width_cm']}x{package['height_cm']}cm, "
            f"volume: {package['volume_cm3']}cm³"
        )

        packages.append(package)

    return packages

# Generate realistic package data
packages_data = generate_realistic_package_data()

# Display the generated packages
print("\n📦 Generated Package Data:")
print("=" * 60)
for i, pkg in enumerate(packages_data, 1):
    print(f"\nPackage {i}:")
    print(f"  Tracking: {pkg['tracking_number']}")
    print(f"  Type: {pkg['package_type']} via {pkg['carrier']}")
    print(f"  Route: {pkg['origin_city']}, {pkg['origin_state']} → {pkg['destination_city']}, {pkg['destination_state']}")
    print(f"  Status: {pkg['current_status']}")
    print(f"  Weight: {pkg['weight_kg']}kg, Volume: {pkg['volume_cm3']}cm³")
    print(f"  Signature Required: {pkg['signature_required']}")

print("\n" + "=" * 60)

# Store each package
for i, pkg in enumerate(packages_data):
    try:
        print(f"\n📦 Storing package {i+1}/3: {pkg['tracking_number']}")
        print(f"   Route: {pkg['origin_city']}, {pkg['origin_state']} → {pkg['destination_city']}, {pkg['destination_state']}")

        # Convert weight from kg to grams (to avoid decimals in Solidity)
        weight_grams = int(pkg['weight_kg'] * 1000)

        # Convert volume to integer
        volume_cm3 = int(pkg['volume_cm3'])

        # Convert estimated delivery to timestamp
        estimated_delivery_timestamp = datetime_to_timestamp(pkg['estimated_delivery'])

        # Create origin and destination strings
        origin_location = f"{pkg['origin_city']}, {pkg['origin_state']}"
        destination_location = f"{pkg['destination_city']}, {pkg['destination_state']}"

        # First estimate gas needed
        try:
            estimated_gas = contract.functions.storePackage(
                pkg['tracking_number'],
                pkg['package_type'],
                pkg['carrier'],
                pkg['current_status'],
                weight_grams,
                volume_cm3,
                origin_location,
                destination_location,
                estimated_delivery_timestamp,
                pkg['signature_required']
            ).estimate_gas({'from': sender_account})

            # Add 20% buffer to estimated gas
            gas_limit = int(estimated_gas * 1.2)
            print(f"📊 Estimated gas: {estimated_gas}, Using: {gas_limit}")

        except Exception as gas_error:
            print(f"⚠️ Gas estimation failed: {gas_error}")
            gas_limit = 500_000  # Fallback to higher limit

        # Store the package with higher gas limit
        tx = contract.functions.storePackage(
            pkg['tracking_number'],        # tracking number
            pkg['package_type'],           # package type
            pkg['carrier'],                # carrier
            pkg['current_status'],         # current status
            weight_grams,                  # weight in grams
            volume_cm3,                    # volume in cubic cm
            origin_location,               # origin city
            destination_location,          # destination city
            estimated_delivery_timestamp,  # estimated delivery timestamp
            pkg['signature_required']      # signature required
        ).transact({
            "from": sender_account,
            "gas": gas_limit,
            "gasPrice": web3.to_wei("2", "gwei")  # Slightly higher gas price
        })

        print(f"📝 Transaction sent: {tx.hex()}")

        # Wait for transaction to be mined
        receipt = web3.eth.wait_for_transaction_receipt(tx)
        print(f"✅ Package {pkg['tracking_number']} stored successfully!")
        print(f"📊 Gas used: {receipt.gasUsed}")

    except Exception as e:
        print(f"❌ Error storing package {pkg['tracking_number']}: {e}")

# Verify all packages were stored
try:
    total_packages = contract.functions.getTotalPackages().call()
    print(f"\n🎯 Total packages stored: {total_packages}")

    # Test retrieving one of the packages
    if total_packages > 0:
        test_tracking = packages_data[0]['tracking_number']
        print(f"\n📋 Testing package retrieval for '{test_tracking}':")
        package_data = contract.functions.getPackage(test_tracking).call()
        print(f"Tracking Number: {package_data[0]}")
        print(f"Package Type: {package_data[1]}")
        print(f"Carrier: {package_data[2]}")
        print(f"Status: {package_data[3]}")
        print(f"Weight (grams): {package_data[4]}")
        print(f"Volume (cm³): {package_data[5]}")
        print(f"Origin: {package_data[6]}")
        print(f"Destination: {package_data[7]}")

        # Convert timestamp back to readable date
        if len(package_data) > 8 and package_data[8] > 0:
            delivery_date = datetime.datetime.fromtimestamp(package_data[8])
            print(f"Estimated Delivery: {delivery_date.strftime('%Y-%m-%d %H:%M:%S')}")

        if len(package_data) > 9:
            print(f"Signature Required: {package_data[9]}")

except Exception as e:
    print(f"❌ Error reading stored packages: {e}")

print(f"\n🎉 Package storage process completed!")

Available accounts:
  [0] 0x39915331974383852B24D36d12a1712A45D50070 - Balance: 99.99706492671484375 ETH
  [1] 0xE8128dc4B3Ac6bd4Bd04A269fd9581A0214ADD17 - Balance: 100 ETH
  [2] 0x8F528ec9F5Da08E76E48e514BC9C594987b1E171 - Balance: 100 ETH
  [3] 0xCb6e113F47f90c86678c4CebD171a1F4a9316218 - Balance: 100 ETH
  [4] 0x133C57ef5ba870bB72C2E8fbC9146115C3699b0D - Balance: 100 ETH
  [5] 0x0bC9Dd331c4A0308A1698bD15AeEa23370F5e37E - Balance: 100 ETH
  [6] 0x79969844C0789F7bc616F03335cE003263Ad125b - Balance: 100 ETH
  [7] 0x4A6c4A67E864C3FAFBAEA4744b74193BC753cbA8 - Balance: 100 ETH
  [8] 0xA71CbFca6e22189C96D00a131e9dF63B95737833 - Balance: 100 ETH
  [9] 0xc3Ce3305fed956488E3222B42820b6D3BC44505A - Balance: 100 ETH

▶️ Using account: 0x39915331974383852B24D36d12a1712A45D50070
💰 Account balance: 99.99706492671484375 ETH

📦 Generated Package Data:

Package 1:
  Tracking: FED486133835
  Type: Overnight via FedEx
  Route: Chicago, California → Virginia Beach, Wisconsin
  Status: Processing
  Weigh

5. Verify it went through

After mining, call again to see the updated count and fetch the index.

In [5]:
# Cell 5: Verify storage with table display
import pandas as pd

# Get verification data
new_total_packages = contract.functions.getTotalPackages().call()
first_package = contract.functions.getPackage("AMA145535234").call()

# Create verification summary table
verification_data = {

    'Metric': ['Total Packages', 'Sample Package ID', 'Package Exists', 'Package Data Length'],
    'Value': [
        new_total_packages,
        'Fed262013051',
        'Yes' if first_package else 'No',
        len(first_package) if first_package else 0
    ]
}

verification_df = pd.DataFrame(verification_data)
print("📊 Storage Verification Summary:")
print("=" * 40)
display(verification_df)

# If package data exists, show detailed package information
if first_package:
    print("\n📦 Sample Package Details:")
    print("=" * 40)

    # Assuming the package data structure (adjust field names as needed)
    package_details = {
        'Field': [
            'trackingNumber',
            'packageType',
            'carrier',
            'currentStatus',
            'weight',
            'volume',
            'originCity',
            'destinationCity',
            'estimatedDelivery',
            'actualDelivery',
            'signatureRequired',
        ],
        'Value': [
            first_package[0] if len(first_package) > 0 else 'N/A',
            first_package[1] if len(first_package) > 1 else 'N/A',
            first_package[2] if len(first_package) > 2 else 'N/A',
            first_package[3] if len(first_package) > 3 else 'N/A',
            first_package[4] if len(first_package) > 4 else 'N/A',
            first_package[5] if len(first_package) > 5 else 'N/A',
            first_package[6] if len(first_package) > 6 else 'N/A',
            first_package[7] if len(first_package) > 7 else 'N/A',
            first_package[8] if len(first_package) > 8 else 'N/A',
            first_package[9] if len(first_package) > 9 else 'N/A',
            first_package[10] if len(first_package) > 10 else 'N/A'
        ]
    }

    package_df = pd.DataFrame(package_details)
    display(package_df)
else:
    print("\n⚠️ No package data found for ID: Fed262013051")

📊 Storage Verification Summary:


,Metric,Value
0,Total Packages,3
1,Sample Package ID,Fed262013051
2,Package Exists,Yes
3,Package Data Length,11



📦 Sample Package Details:


,Field,Value
0,trackingNumber,AMA145535234
1,packageType,Fragile
2,carrier,Amazon Logistics
3,currentStatus,Out for Delivery
4,weight,15480
5,volume,228143
6,originCity,"Minneapolis, North Carolina"
7,destinationCity,"Philadelphia, Utah"
8,estimatedDelivery,1749969000
9,actualDelivery,0


In [6]:
# Cell 6: Adding New IoT Data using our Output (logistics_data.json) on the Blockchain
import pandas as pd
import json

# Load the JSON data
with open("../data/logistics_data.json", "r") as file:
    logistics_data = json.load(file)

# Convert to DataFrame for easier handling (if it's a list of JSON objects)
if isinstance(logistics_data, list):
    df = pd.DataFrame(logistics_data)
else:
    # If it's a single JSON object, convert to list first
    df = pd.DataFrame([logistics_data])

sender_account = web3.eth.accounts[0]
print(f"🔐 Using account: {sender_account}")
print("=" * 50)

# Create a list to store transaction results for table display
transaction_results = []

for i, row in df.iterrows():
    try:
        # Get tracking number (adjust field name based on your JSON structure)
        tracking_id = row.get('trackingNumber', row.get('tracking_number', f'package_{i}'))

        # Convert entire row to JSON string for storage
        row_json = row.to_json()

        # Extract data with proper error handling and type conversion
        tracking_number = str(row.get('tracking_number', row.get('trackingNumber', f'package_{i}')))
        package_type = str(row.get('package_type', row.get('packageType', 'unknown')))
        carrier = str(row.get('carrier', 'unknown'))
        status = str(row.get('status', row.get('currentStatus', 'unknown')))

        # Handle weight - check multiple possible locations
        weight = 0
        if 'weight' in row:
            weight = int(float(row['weight']))
        elif 'dimensions' in row and isinstance(row['dimensions'], dict) and 'weight' in row['dimensions']:
            weight = int(float(row['dimensions']['weight']))

        # Handle volume - check multiple possible locations
        volume = 0
        if 'volume' in row:
            volume = int(float(row['volume']))
        elif 'dimensions' in row and isinstance(row['dimensions'], dict) and 'volume' in row['dimensions']:
            volume = int(float(row['dimensions']['volume']))

        # Handle origin city
        origin_city = 'unknown'
        if 'originCity' in row:
            origin_city = str(row['originCity'])
        elif 'origin' in row and isinstance(row['origin'], dict) and 'city' in row['origin']:
            origin_city = str(row['origin']['city'])

        # Handle destination city
        destination_city = 'unknown'
        if 'destinationCity' in row:
            destination_city = str(row['destinationCity'])
        elif 'destination' in row and isinstance(row['destination'], dict) and 'city' in row['destination']:
            destination_city = str(row['destination']['city'])

        # Handle delivery dates - convert to timestamp if needed
        estimated_delivery = 0  # Default timestamp
        if 'estimated_delivery' in row or 'estimatedDelivery' in row:
            delivery_str = row.get('estimated_delivery', row.get('estimatedDelivery', ''))
            # If it's already a number, use it; otherwise convert or use 0
            if isinstance(delivery_str, (int, float)):
                estimated_delivery = int(delivery_str)
            elif delivery_str.isdigit():
                estimated_delivery = int(delivery_str)

        # Convert signature_required to boolean
        signature_required_str = row.get('signature_required')

        # Execute the smart contract transaction
        # Note: Contract expects 10 parameters (no actualDelivery in storePackage)
        tx = contract.functions.storePackage(
            tracking_number,        # string _trackingNumber
            package_type,          # string _packageType
            carrier,               # string _carrier
            status,                # string _currentStatus
            weight,                # uint256 _weight
            volume,                # uint256 _volume
            origin_city,           # string _originCity
            destination_city,      # string _destinationCity
            estimated_delivery,    # uint256 _estimatedDelivery
            signature_required_str     # bool _signatureRequired
        ).transact({
            "from": sender_account,
            "gas": 300_000,
            "gasPrice": web3.to_wei("1", "gwei")
        })

        receipt = web3.eth.wait_for_transaction_receipt(tx)

        # Store result for table
        transaction_results.append({
            'Index': i+1,
            'Tracking ID': tracking_id,
            'Status': '✅ Success',
            'Gas Used': receipt.gasUsed,
            'Transaction Hash': tx.hex()[:12] + '...'  # Shortened for display
        })

        print(f"[{i+1}/{len(df)}] ✅ Stored package: {tracking_id} | Tx: {tx.hex()}")

    except Exception as e:
        # Store error result for table
        transaction_results.append({
            'Index': i+1,
            'Tracking ID': tracking_id if 'tracking_id' in locals() else f'package_{i}',
            'Status': '❌ Failed',
            'Gas Used': 0,
            'Transaction Hash': 'N/A'
        })

        print(f"[{i+1}/{len(df)}] ❌ Error storing package {tracking_id if 'tracking_id' in locals() else f'package_{i}'}: {e}")

# Display results in table format
print("\n📊 Transaction Summary:")
print("=" * 50)
results_df = pd.DataFrame(transaction_results)
display(results_df)

# Display summary statistics
successful_txs = len([r for r in transaction_results if '✅' in r['Status']])
failed_txs = len([r for r in transaction_results if '❌' in r['Status']])
total_gas_used = sum([r['Gas Used'] for r in transaction_results])

summary_data = {
    'Metric': ['Total Packages Processed', 'Successful Transactions', 'Failed Transactions', 'Success Rate', 'Total Gas Used'],
    'Value': [
        len(df),
        successful_txs,
        failed_txs,
        f"{(successful_txs/len(df)*100):.1f}%" if len(df) > 0 else "0%",
        f"{total_gas_used:,}"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n📈 Processing Summary:")
print("=" * 50)
display(summary_df)

🔐 Using account: 0x39915331974383852B24D36d12a1712A45D50070
[1/200] ✅ Stored package: UPS212189171 | Tx: 78c26c6cedf371e2179af26eebc71226e79b00139f83e6a09b501fa725972b2c
[2/200] ✅ Stored package: Fed693941012 | Tx: 619744b92f137972ed01cde3d1e06850c091e1b2e9eed6c2f6518f9c46548412
[3/200] ✅ Stored package: DHL359638010 | Tx: e409fe15ca7d30c8023f9c1b597adcd163ac0dcea2c73fa0a50b87bfc154ea68
[4/200] ✅ Stored package: UPS861730847 | Tx: 70a86905488f6cac59b09533764fe255e240ac1637c52d98df1212bcfdd39b48
[5/200] ✅ Stored package: USP228720101 | Tx: 219abfe3c98aca9725600984e35c71848e23ea27036c4f1d09a6d6c0abd5521c
[6/200] ✅ Stored package: DHL829612681 | Tx: 128ec632e890f5d2309a4d4aea54fc19ebb180cbe4ae1b1aa16086f25fe58241
[7/200] ✅ Stored package: DHL784748644 | Tx: fbf76d7a428fb582bb67e4f52f6d8ea51fa0eac48a30eba632404866f8eb04b0
[8/200] ✅ Stored package: UPS444530304 | Tx: d1b536c85f54f922041ee5f9a077a091aab5c1c108ff2ec991de1b414797b363
[9/200] ✅ Stored package: USP11532327 | Tx: f0e455a1aebee721

,Index,Tracking ID,Status,Gas Used,Transaction Hash
0,1,UPS212189171,✅ Success,275568,78c26c6cedf3...
1,2,Fed693941012,✅ Success,295480,619744b92f13...
2,3,DHL359638010,✅ Success,295396,e409fe15ca7d...
3,4,UPS861730847,✅ Success,295432,70a86905488f...
4,5,USP228720101,✅ Success,295612,219abfe3c98a...
...,...,...,...,...,...
195,196,Ama510359243,✅ Success,275712,5e82817f523d...
196,197,USP159901721,✅ Success,275544,6780c0acc3be...
197,198,Ama249766800,✅ Success,275724,2b9c3b63eff9...
198,199,Ama126697393,✅ Success,275820,27513211dda1...



📈 Processing Summary:


,Metric,Value
0,Total Packages Processed,200
1,Successful Transactions,200
2,Failed Transactions,0
3,Success Rate,100.0%
4,Total Gas Used,"56,579,224"


In [7]:
# Read first and last package records from blockchain

def display_package_info(package_data, record_type):
    """Helper function to display package information in a formatted way"""
    field_names = [
        'trackingNumber', 'packageType', 'carrier', 'currentStatus',
        'weight', 'volume', 'originCity', 'destinationCity',
        'estimated_delivery', 'signature_required'
    ]

    print(f"\n📦 {record_type} Package Record:")
    print("=" * 60)

    for i, field_name in enumerate(field_names):
        value = package_data[i]

        # Format specific fields for better readability
        if field_name in ['estimated_delivery', 'signature_required']:
            if value == 0:
                formatted_value = "Not set"
            else:
                # Convert timestamp to readable date (if it's a timestamp)
                try:
                    formatted_value = datetime.fromtimestamp(value).strftime('%Y-%m-%d %H:%M:%S')
                    print("It should be like this" + formatted_value)
                except:
                    formatted_value = str(value)
        elif field_name == 'signature_required':
            formatted_value = "Yes" if value else "No"
        elif field_name in ['weight', 'volume']:
            formatted_value = f"{value:,}" if value > 0 else "Not specified"
        else:
            formatted_value = value if value else "Not specified"

        print(f"{field_name:18}: {formatted_value}")

try:
    # Get total number of packages stored
    total_packages = contract.functions.getTotalPackages().call()
    print(f"📊 Total packages stored on blockchain: {total_packages}")

    if total_packages == 0:
        print("❌ No packages found on the blockchain!")
    else:
        # Get the first package (index 0)
        try:
            first_package = contract.functions.packages(0).call()
            display_package_info(first_package, "FIRST")
        except Exception as e:
            print(f"❌ Error reading first package: {e}")
            # Alternative method: try to get package by tracking number if you know it
            print("💡 Try using getPackage() with a known tracking number instead")

        # Get the last package (index total_packages - 1)
        if total_packages > 1:
            try:
                last_package = contract.functions.packages(total_packages - 1).call()
                display_package_info(last_package, "LAST")
            except Exception as e:
                print(f"❌ Error reading last package: {e}")
        else:
            print("\n📝 Only one package stored - first and last are the same")

        # Additional info: Show total events stored
        try:
            total_events = contract.functions.getTotalEvents().call()
            print(f"\n📈 Total tracking events stored: {total_events}")
        except Exception as e:
            print(f"❌ Error getting total events: {e}")

except Exception as e:
    print(f"❌ Error accessing blockchain contract: {e}")
    print("\n💡 Alternative approach - if you know specific tracking numbers:")
    print("Example:")
    print("package_info = contract.functions.getPackage('TRACK123').call()")
    print("display_package_info(package_info, 'SPECIFIC')")

📊 Total packages stored on blockchain: 203

📦 FIRST Package Record:
trackingNumber    : FED486133835
packageType       : Overnight
carrier           : FedEx
currentStatus     : Processing
weight            : 25,780
volume            : 494,628
originCity        : Chicago, California
destinationCity   : Virginia Beach, Wisconsin
estimated_delivery: 1749969000
signature_required: Not set

📦 LAST Package Record:
trackingNumber    : UPS102825962
packageType       : International
carrier           : UPS
currentStatus     : unknown
weight            : 36
volume            : 35,483
originCity        : North Las Vegas
destinationCity   : New York
estimated_delivery: Not set
signature_required: Not set

📈 Total tracking events stored: 0


# Data Retrieval and Processing

In [16]:
total_package_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_package_records}")

Total IoT records stored: 101


In [12]:
# Cell: Retrieve and Display All Tracking Events (IoT Records)
import pandas as pd
from datetime import datetime

# Step 1: Retrieve total number of tracking events
try:
    total_events = contract.functions.getTotalEvents().call()
except Exception as e:
    print(f"❌ Error retrieving total events: {e}")
    total_events = 0

# Step 2: Fetch all tracking events
event_data = []

if total_events > 0:
    for i in range(total_events):
        try:
            event = contract.functions.getTrackingEvent(i).call()
            event_data.append({
                "tracking_number": event[0],
                "timestamp": datetime.fromtimestamp(event[1]).strftime('%Y-%m-%d %H:%M:%S'),
                "status": event[2],
                "location": event[3],
                "description": event[4],
                "scan_type": event[5],
                "operator_id": event[6]
            })
        except Exception as e:
            print(f"⚠️ Error retrieving event #{i}: {e}")

# Step 3: Display summary
verification_data = {
    'Metric': ['Total Events on Chain', 'Successfully Retrieved', 'Errors Encountered'],
    'Value': [
        total_events,
        len(event_data),
        total_events - len(event_data)
    ]
}

verification_df = pd.DataFrame(verification_data)
print("📊 Tracking Events Retrieval Summary:")
print("=" * 40)
display(verification_df)

# Step 4: Display full event table
if event_data:
    events_df = pd.DataFrame(event_data)
    print("\n📟 Full Tracking Events Table:")
    print("=" * 40)
    display(events_df)
else:
    print("\n⚠️ No tracking event data available.")


📊 Tracking Events Retrieval Summary:


,Metric,Value
0,Total Events on Chain,0
1,Successfully Retrieved,0
2,Errors Encountered,0



⚠️ No tracking event data available.


In [15]:
# Cell: Clean and Display IoT Data Values
import numpy as np
import pandas as pd

# Step 1: Ensure `events_df` exists (from previous trackingEvents retrieval)
if 'events_df' in locals() and not events_df.empty:

    print("🧹 Cleaning IoT Data Values:")
    print("=" * 40)

    # Step 2: Extract numeric values from 'description' (assuming data_value stored there)
    events_df["numeric_value"] = events_df["description"].str.extract(r'(\d+\.?\d*)')

    # Step 3: Convert to float and handle errors
    events_df["numeric_value"] = pd.to_numeric(events_df["numeric_value"], errors='coerce')

    # Step 4: Fill missing numeric values with 0
    events_df["numeric_value"].fillna(0, inplace=True)

    # Step 5: Summary statistics
    numeric_summary = {
        'Metric': ['Total Records', 'Non-zero Numeric Entries', 'Missing Before Fill'],
        'Value': [
            len(events_df),
            (events_df["numeric_value"] > 0).sum(),
            events_df["numeric_value"].isna().sum()
        ]
    }

    summary_df = pd.DataFrame(numeric_summary)
    display(summary_df)

    # Step 6: Display cleaned table
    print("\n📟 Cleaned IoT Data Table (Preview):")
    print("=" * 40)
    display(events_df.head())

else:
    print("⚠️ No IoT data found to clean. Please run the event retrieval step first.")


⚠️ No IoT data found to clean. Please run the event retrieval step first.
